In [ ]:
# sagemath version 9.8
# Author : E. Poimenidou

In [47]:
reset()
import numpy as np
import networkx as nx
from numpy import linalg as LA
import random
from fpylll import IntegerMatrix,LLL,GSO
from fpylll.util import gaussian_heuristic

# uses fpylll to return the gram schimdt matrix and the ||bi*||


def find_k_and_P(q):
    for k in range(1,200):
        A,B,C = -(1+k^2), -k^2+2*k*q-1, q*(k-q)
        g = A*x^2+B*x+C
        P = floor( k*q / (k^2+1)  )
        G=g.subs(x=P)
        if G>0:
            A,B,C = -(1+(k-1)^2), -(k-1)^2+2*(k-1)*q-1, q*(k-1-q)
            g = A*x^2+B*x+C
            P = floor( (k-1)*q / ( (k-1)^2+1)  )
            G=g.subs(x=P)
            print(q,":",k-1,P)  
            return k-1,P
        
def gram_schimdt(A, n):
    M = GSO.Mat(A)
    M.update_gso()
    rows = A.nrows
    matrix = np.array([M.get_mu(i, j) for i in range(rows) for j in range(rows)]).reshape(rows, rows)
    orth_norm = np.array([M.get_r(k, k) for k in range(n)])
    return matrix, orth_norm

def mat2fp(A):
    L = IntegerMatrix(A.shape[0], A.shape[1])
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            L[i, j] = int(A[i, j])
    return L

def get_qij(superbasis):
    n = len(superbasis)
    Q = [superbasis[i].dot_product(superbasis[j]) for i in range(n) for j in range(n)]
    Q = matrix(n, n, Q)
    return Q

def get_superbasis(basis): # basis is a list
    n = len(basis)
    basis = [vector(x) for x in basis]
    superbasis_vector = -sum(basis[i] for i in [0..n-1])
    return [vector(basis[i]) for i in [0..n-1]] + [vector(superbasis_vector)]

def length(v):
    n = len(v[0])
    x=0
    print(n)
    for i in range(n):
        x=x+v[0][i]**2
    return x.expand()

def t_coeffs(dimension, set_c, source):
    set_c.remove(source)
    t = np.zeros(shape=dimension)
    for i in set_c:
        t[i-1] = 1
    return t

def do_mincut_svp(n, Q):
    G = create_weighted_graph_networkx_svp(n, Q) 
    source = 0
    sink = n
    cut_value, partition = nx.minimum_cut(G, source, sink)
    reachable, non_reachable = partition
    cut = reachable if source in reachable else non_reachable
    t_c = t_coeffs(n+1,cut,0)
    return t_c, cut_value


def create_weighted_graph_networkx_svp(dimension, Q):
    """
    source and sink are two positive integers, source = 0 and sink = n+2
    S=(s_i) is given by s_i=-2sum_{j} q_{ij}p_j
    Graph has n+3 nodes: the 1st node is the source = 0, the next n+1 nodes represent each vector 
    of the superbasis and thelast n+3 th node is the sink
    """
    graph = nx.Graph()
    source = 0
    sink = dimension
    graph.add_nodes_from(range(source, sink))

    for i in range(source, sink+1):
        for j in range(source, sink+1):
            graph.add_edge(i, j, capacity=-Q[i][j])
    return graph

def is_vfk(Q,N):
    '''
    Q is the Gram-Schimdt matrix of the superbasis
    N is the parameter given when we build ntru_vfk_matrix
    '''
    lst=[]
    x=np.array([Q])
    lst=x[np.nonzero(x)]
    lst=list(lst)
    for i in range(2*N+1):
        if Q[i][i]!=0:
            lst.remove(Q[i][i])
    return all(val <= 0 for val in lst)

def length(v):
    n = len(v[0])
    x=0
    print(n)
    for i in range(n):
        x=x+v[0][i]**2
    return x.expand()

def svp_vfk(N,Q,super_basis):
    t, weight = do_mincut_svp(2*N,Q)
    vec = np.dot(t, super_basis).astype(int)
    print("length :", np.linalg.norm(vec))
    print("vector=",vec)
    return vec

In [53]:
def matrix_for_the_lattice(N,q,F):
    H = matrix(N)
    I=identity_matrix(N)

    Zero_Matrix=matrix(N)

    for i in range(N):
        for j in range(N):
            H[i,j] = Convolution_in_R(Zx(F),x^i,N)[j]
       
    B_1=block_matrix([[I,H]])       
    B_2=block_matrix([[Zero_Matrix,q*I]])
    M_NTRU=block_matrix([[B_1],[B_2]])
    return M_NTRU

def init_attack(N,q):
    kappa,P=find_k_and_P(q)
    alpha_vector_vfk = [-kappa] + [0 for i in range(N-1)] 
    A = Zx(alpha_vector_vfk)
    M_k = matrix_for_the_lattice(N,q,A)
    M_NTRU_VFK=unimodular(P,N)*M_k
    # from paper: remark 4.1
    r = - ((kappa*P + kappa - q)*kappa + P + 1)
    s = (kappa*P + kappa - q)*(q - kappa*P ) - (P + 1)*P
    t = (kappa*P-q)*kappa + P
    diag1 = N * ( (1 + P )^2 + ( kappa*(P + 1) - q)^2 )
    diag2 = 1+kappa^2
    basis = M_NTRU_VFK.rows()
    super_basis=get_superbasis(basis)
    Q=get_qij(super_basis)
    print("is VFK?",is_vfk(Q,N))
    return kappa,Q,M_NTRU_VFK,M_k,super_basis

def unimodular(P,N):
    '''
    N,P see the paper (both are positive integer)
    The output is a unimodular matrix of dimension 2N x 2N
    '''
    #upper block
    I=identity_matrix(N)
    Zero_Matrix=matrix(N)
    #lower block
    matrix_P = P*I
    
    B_u = block_matrix([[I,Zero_Matrix]])   
    B_l = block_matrix([[matrix_P,I]])  
    unimod_matrix = block_matrix([[B_u],[B_l]])   
    return unimod_matrix

def Convolution_in_R(f,g,N):
    return (f*g)%(x^N-1)
Zx.<x> = ZZ[]

In [77]:
#ntruhps2048509
N,q=509,2048
kappa,Q,M_NTRU_VFK,M_k,super_basis=init_attack(N,q)
result = svp_vfk(N,Q,super_basis)

2048 : 64 31
is VFK? True
length : 64.00781202322104
vector= [  0   0   0 ...   0   0 -64]


In [78]:
#ntruhps2048677
N,q=677,2048
kappa,Q,M_NTRU_VFK,M_k,super_basis=init_attack(N,q)
result = svp_vfk(N,Q,super_basis)

2048 : 64 31
is VFK? True
length : 64.00781202322104
vector= [  0   0   0 ...   0   0 -64]


In [79]:
#sntrup653
N,q=653,4621
kappa,Q,M_NTRU_VFK,M_k,super_basis=init_attack(N,q)
result = svp_vfk(N,Q,super_basis)

4621 : 101 45
is VFK? True
length : 101.00495037373169
vector= [   0    0    0 ...    0    0 -101]


In [80]:
#sntrup761
N,q=761,4591
kappa,Q,M_NTRU_VFK,M_k,super_basis=init_attack(N,q)
result = svp_vfk(N,Q,super_basis)

4591 : 98 46
is VFK? True
length : 98.00510190801293
vector= [  0   0   0 ...   0   0 -98]
